## Dependencies


In [1]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
os.chdir('../')
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [2]:
%pwd

'c:\\Projects\\python\\brain_tumor_segmentation'

In [3]:
train_path = Path('.\data\\brain-tumor-mri-dataset\Training')
test_path = Path('.\data\\brain-tumor-mri-dataset\Testing')

## Data loader

In [7]:
transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
    # transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

train_dataset = ImageFolder(root=train_path,transform=transform)
valid_size = int(len(train_dataset)*0.15)
train_size = len(train_dataset)-valid_size
test_data = ImageFolder(root=test_path,transform=transform)
train_data,valid_data = random_split(train_dataset,[train_size,valid_size])

train_loader = DataLoader(dataset=train_data,batch_size=32,shuffle=True)
valid_loader = DataLoader(dataset=valid_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset=test_data,batch_size=32,shuffle=True)

In [12]:
for image,label in train_loader:
    print(image.shape,label.shape)
    print(label)
    break

torch.Size([32, 3, 256, 256]) torch.Size([32])
tensor([1, 3, 3, 2, 1, 3, 0, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 2, 2, 0, 2, 3, 3, 2,
        3, 1, 2, 2, 1, 3, 1, 2])
